# Modeling and Simulation in Python

Case study: Throwing Axe

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [4]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

### Throwing axe

Our favorite event at Lumberjack Competitions is axe throwing.  The axes used for this event typically weigh 1.5 to 2 kg, with handles roughly 0.7 m long.  They are thrown overhead at a target typically 6 m away and 1.5 m off the ground.  Normally, the axe makes one full rotation in the air to hit the target blade first, with the handle close to vertical.

![Diagram of throwing axe](diagrams/throwingaxe1.png)

Here's a version of `make_system` that sets the initial conditions.

The state variables are x, y, theta, vx, vy, omega, where theta is the orientation (angle) of the axe in radians and omega is the angular velocity in radians per second.

I chose initial conditions based on videos of axe throwing.

In [13]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
radian = UNITS.radian

def make_system():
    """Makes a System object for the given conditions.
    
    returns: System with init, ...
    """
    P = Vector(0, 2) * m
    V = Vector(8, 4) * m/s
    
    init = State(x=P.x, y=P.y, theta=2, 
                 vx=V.x, vy=V.y, omega=-7)

    t_end = 1.0 * s
    
    return System(init=init, t_end=t_end,
                  g = 9.8 * m/s**2,
                  mass = 1.5 * kg,
                  length = 0.7 * m)

Let's make a `System`

In [14]:
system = make_system()

In [15]:
system.init

As a simple starting place, I ignore drag, so `vx` and `omega` are constant, and `ay` is just `-g`.

In [16]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with length0, m, k
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, theta, vx, vy, omega = state
    unpack(system)

    ax = 0
    ay = -g
    alpha = 0

    return vx, vy, omega, ax, ay, alpha

As always, let's test the slope function with the initial conditions.

In [17]:
slope_func(system.init, 0, system)

And then run the simulation.

In [22]:
results, details = run_ode_solver(system, slope_func, max_step=0.05)
details

In [23]:
results.tail()

### Visualizing the results

The simplest way to visualize the results is to plot the state variables as a function of time.

In [26]:
plot(results.x, label='x')
plot(results.y, label='y')

decorate(xlabel='Time (s)',
         ylabel='Position (m)')

In [27]:
plot(results.theta, label='theta', color='C2')

decorate(xlabel='Time (s)',
         ylabel='Angle (radian)')

We can plot the velocities the same way.

In [31]:
plot(results.vx, label='vx')
plot(results.vy, label='vy')

decorate(xlabel='Time (s)',
         ylabel='Velocity (m/s)')

In [32]:
plot(results.omega, label='omega', color='C2')

decorate(xlabel='Time (s)',
         ylabel='Angular velocity (rad/s)')

Another way to visualize the results is to plot y versus x.  The result is the trajectory through the plane of motion.

In [33]:
plot(results.x, results.y, label='trajectory', color='C3')

decorate(xlabel='x position (m)',
         ylabel='y position (m)')

## Exercises

**Exercise:**  Find the starting conditions that make the final height of the COG as close as possible to 1.5 m.  Ideally, the final angle should be a little past vertical.

**Exercise:**  Compute the total velocity of the leading edge of the axe at the point of impact, that is, the sum of velocity due to translation and rotation.

In [34]:
def make_frame(theta):
    rhat = Vector(pol2cart(theta, 1))
    that = rhat.perp()
    return rhat, that

In [38]:
# quick, get those state variables into Vector objects!
state = get_last_value(results)
x, y, theta, vx, vy, omega = state
P = Vector(x, y)
V = Vector(vx, vy)
rhat, that = make_frame(theta)

In [40]:
# Solution goes here

## Animation

NOTE: This section needs to be updated.

Animating this system is a little more complicated, if we want to show the shape and orientation of the axe.

It is useful to construct a frame with $\hat{r}$ along the handle of the axe and $\hat{\theta}$ perpendicular.

Now we're ready to animate the results.  The following figure shows the frame and the labeled points A, B, C, and D.

![Diagram of the axe with reference frame](diagrams/throwingaxe2.png)

In [14]:
decorate(xlabel='x position (m)',
         ylabel='y position (m)',
         xlim=[0, 8.1],
         ylim=[0, 5.5],
         legend=False)



for t, state in system.results.iterrows():
    x, y, theta, vx, vy, omega = state
    P = Vector(x, y)
    rhat, that = make_frame(theta)
    
    # plot the handle
    A = P - l1 * rhat
    B = P + l2 * rhat
    plot_segment(A, B, color='red', update=True)

    # plot the axe head
    C = B + l2 * that
    D = B - l2 * that
    plot_segment(C, D, color='black', linewidth=10, update=True)

    # plot the COG
    plot(x, y, 'bo', update=True)
    sleep(0.01)

During the animation, the parts of the axe seem to slide around relative to each other.  I think that's because the lines and circles get rounded off to the nearest pixel.

Here's the final state of the axe at the point of impact (assuming the target is 8 m away).

In [15]:
state